# Реализация алгоритма run-length encoding

In [169]:
import random
import string
import struct
import os

## 1. Реализация

In [170]:
def zip_rle(filename):
    with open(filename, 'rb') as fr, open(filename + '.rle', 'wb+') as fw:
        
        # позиция указателя для записи в файл подсчитанного количества элементов
        counter_pos = 0
        # позиция указателя для записи в файл элемента
        element_pos = 0
        # рассматриваем элементы в исходном файле тройками: предыдущий, текущий, следующий
        prev_el = None
        curr_el = fr.read(1)
        
        while True:
            next_el = fr.read(1)
            # задаем начальные значения
            if prev_el is None:
                if curr_el == next_el:
                    counter = 1
                else:
                    counter = -1
                element_pos += 1
                fw.seek(element_pos)
                fw.write(curr_el)
                element_pos += 1
                
            if prev_el is not None:
                # aAb->aAb
                # aAa->aAa
                if prev_el == curr_el:
                    if counter < 127:
                        counter += 1
                    else:
                        fw.seek(counter_pos)
                        fw.write(struct.pack('b', counter))
                        counter_pos = element_pos
                        if curr_el == next_el:
                            counter = 1
                        else:
                            counter = -1
                        counter_pos = element_pos
                        element_pos += 1
                        fw.seek(element_pos)
                        fw.write(curr_el)
                        element_pos += 1
                if prev_el != curr_el:
                    # bAa->b Aa
                    if curr_el == next_el:
                        fw.seek(counter_pos)
                        fw.write(struct.pack('b', counter))
                        counter_pos = element_pos
                        counter = 1
                        element_pos += 1
                    if curr_el != next_el:
                        # aBc->aBc
                        if counter < 0:
                            if counter > -127:
                                counter -= 1
                            else:
                                fw.seek(counter_pos)
                                fw.write(struct.pack('b', counter))
                                counter_pos = element_pos
                                if curr_el == next_el:
                                    counter = 1
                                else:
                                    counter = -1
                                counter_pos = element_pos
                                element_pos += 1
                        # aaBc->aa Bc
                        else:
                            fw.seek(counter_pos)
                            fw.write(struct.pack('b', counter))
                            counter_pos = element_pos
                            counter = -1
                            element_pos += 1
                    fw.seek(element_pos)
                    fw.write(curr_el)
                    element_pos += 1

            if next_el == b'':
                fw.seek(counter_pos)
                fw.write(struct.pack('b', counter))
                break

            prev_el, curr_el = curr_el, next_el

In [171]:
def unzip_rle(filename_in, filename_out):
    with open(filename_in, 'rb') as fr, open(filename_out, 'wb') as fw:
        while True:
            data = fr.read(1)
            if not data:
                break
            counter = struct.unpack('b', data)[0]
            if counter < 0:
                for i in range(abs(counter)):
                    element = fr.read(1)
                    fw.write(element)
            if counter > 0:
                element = fr.read(1)
                for i in range(counter):
                    fw.write(element)

## 2. Тестирование

In [172]:
def generate_test_file(filename, n, m=1):
    with open(filename, 'w') as fw:
        for i in range(n):
            element = random.choice(string.ascii_lowercase)
            for i in range(m):
                fw.write(element)
                
def compare_files(filename_1, filename_2):                   
    with open(filename_1, 'r') as fr, open(filename_2, 'r') as fo:
        print(fr.read() == fo.read())

In [173]:
filename_init = 'test'
filename_zipped = filename_init + '.rle'
filename_unzipped = 'test_unzipped'

### Тест №1

- Текстовый файл
- 26 различных символов
- Случайный порядок

In [174]:
generate_test_file(filename_init, 10000)
zip_rle(filename_init)
unzip_rle(filename_zipped, filename_unzipped)

In [175]:
compare_files(filename_init, filename_unzipped)

True


In [176]:
print('Размер исходного файла:', os.stat(filename_init).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init).st_size / os.stat(filename_zipped).st_size, 2))

Размер исходного файла: 10000
Размер сжатого файла: 10363
Коэффициент сжатия: 0.96


### Тест №2

- Текстовый файл
- 26 различных символов
- Каждый символ задублирован

In [177]:
generate_test_file(filename_init, 10000, m=2)
zip_rle(filename_init)
unzip_rle(filename_zipped, filename_unzipped)

In [178]:
compare_files(filename_init, filename_unzipped)

True


In [179]:
print('Размер исходного файла:', os.stat(filename_init).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init).st_size / os.stat(filename_zipped).st_size, 2))

Размер исходного файла: 20000
Размер сжатого файла: 19260
Коэффициент сжатия: 1.04


### Тест №3

- Текстовый файл
- 26 различных символов
- Каждый символ повторяется 150 раз

In [180]:
generate_test_file(filename_init, 1000, m=150)
zip_rle(filename_init)
unzip_rle(filename_zipped, filename_unzipped)

In [181]:
compare_files(filename_init, filename_unzipped)

True


In [182]:
print('Размер исходного файла:', os.stat(filename_init).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init).st_size / os.stat(filename_zipped).st_size, 2))

Размер исходного файла: 150000
Размер сжатого файла: 3922
Коэффициент сжатия: 38.25


In [183]:
os.remove(filename_init)
os.remove(filename_zipped)
os.remove(filename_unzipped)

### Тест №4

- Изображение

In [156]:
filename_init_img = 'image.png'
filename_zipped_img = filename_init_img + '.rle'

In [157]:
zip_rle(filename_init_img)

In [158]:
print('Размер исходного файла:', os.stat(filename_init_img).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped_img).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init_img).st_size / os.stat(filename_zipped_img).st_size, 2))

Размер исходного файла: 1364789
Размер сжатого файла: 1380456
Коэффициент сжатия: 0.99


### Тест №5

- Звуковой файл

In [159]:
filename_init_a1 = 'wave.mp3'
filename_zipped_a1 = filename_init_a1 + '.rle'

In [160]:
zip_rle(filename_init_a1)

In [161]:
print('Размер исходного файла:', os.stat(filename_init_a1).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped_a1).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init_a1).st_size / os.stat(filename_zipped_a1).st_size, 2))

Размер исходного файла: 15103189
Размер сжатого файла: 15191258
Коэффициент сжатия: 0.99


### Тест №6

- Звуковой файл
- Присутствуют моменты без музыки

In [162]:
filename_init_a2 = 'waterfront.mp3'
filename_zipped_a2 = filename_init_a2 + '.rle'

In [163]:
zip_rle(filename_init_a2)

In [164]:
print('Размер исходного файла:', os.stat(filename_init_a2).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped_a2).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init_a2).st_size / os.stat(filename_zipped_a2).st_size, 2))

Размер исходного файла: 8671924
Размер сжатого файла: 8392413
Коэффициент сжатия: 1.03


### Тест №6

- Сжатый архиватором файл

In [165]:
filename_init_zip = 'compressed.zip'
filename_zipped_zip = filename_init_zip + '.rle'

In [166]:
zip_rle(filename_init_zip)

In [167]:
print('Размер исходного файла:', os.stat(filename_init_zip).st_size)
print('Размер сжатого файла:', os.stat(filename_zipped_zip).st_size)
print('Коэффициент сжатия:', round(os.stat(filename_init_zip).st_size / os.stat(filename_zipped_zip).st_size, 2))

Размер исходного файла: 64511
Размер сжатого файла: 65152
Коэффициент сжатия: 0.99


## 3. Выводы

|Тип файла|Коэффициент сжатия|
|:------:|----------|
|Текстовый файл 1|0.97|
|Текстовый файл 2|1.04|
|Текстовый файл 3|38.15|
|Изображение|0.99|
|Аудио 4|0.99|
|Аудио 5|1.03|
|Сжатый|0.99|

Сжатию по алгоритму RLE поддаются файлы, содержащие повторяющиеся элементы. Например: текст с одинаковыми символами, аудио с паузами, изображения с доминирующим цветом. В противном случае получившийся при применении RLE файл получается размером больше исходного, т.к. добавляется еще информация о количестве подсчитанных элементов.